In [8]:
import json

# Step 1: Load the JSON file
with open('F:\\deepfake-detection-challenge\\train_sample_videos\\metadata.json', 'r') as file:
    data = json.load(file)

print(data)

# Step 2: Access the labels
# Assuming the labels are stored under a specific key, for example, 'labels'
labels = {video: details['label'] for video, details in data.items()}

# Print or use the labels as needed
print(labels)

for video, label in labels.items():
    print(f"video: {video} Label: {label}")

{'aagfhgtpmv.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'vudstovrck.mp4'}, 'aapnvogymq.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'jdubbvfswz.mp4'}, 'abarnvbtwb.mp4': {'label': 'REAL', 'split': 'train', 'original': None}, 'abofeumbvv.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'atvmxvwyns.mp4'}, 'abqwwspghj.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'qzimuostzz.mp4'}, 'acifjvzvpm.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'kbvibjhfzo.mp4'}, 'acqfdwsrhi.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'ccfoszqabv.mp4'}, 'acxnxvbsxk.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'fjlyaizcwc.mp4'}, 'acxwigylke.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'ffcwhpnpuw.mp4'}, 'aczrgyricp.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'slwkmefgde.mp4'}, 'adhsbajydo.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'fysyrqfguw.mp4'}, 'adohikbdaz.mp4': {'label': 'FAKE', 'split': 'train', 'original': 'qjlhemtkx

In [9]:
import torch

# Check if CUDA is available and the correct CUDA version is installed
print(f"Is CUDA available: {torch.cuda.is_available()}")
print(f"PyTorch CUDA version: {torch.version.cuda}")
print(f"CUDA Device Name: {torch.cuda.get_device_name(0)}")


Is CUDA available: True
PyTorch CUDA version: 12.4
CUDA Device Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [33]:
import os
import cv2
import numpy as np
from retinaface import RetinaFace
import matplotlib.pyplot as plt
import torch

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Folder containing videos
video_folder = "F:\\deepfake-detection-challenge\\train_sample_videos"  # Replace with your folder path

# List all video files in the folder
video_files = [f for f in os.listdir(video_folder) if f.endswith(('.mp4', '.avi', '.mov'))]

# Frame interval (e.g., process every 10th frame)
frame_interval = 10

# Process each video file
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    print(f"Processing video: {video_file}")
    
    # Load the video
    cap = cv2.VideoCapture(video_path)
    
    # Initialize frame count
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break  # Break the loop if video ends
        
        # Increment frame count
        frame_count += 1
        
        # Skip frames based on the interval
        if frame_count % frame_interval != 0:
            continue
        
        # Convert frame from BGR (OpenCV default) to RGB (RetinaFace expects RGB)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Move frame to the GPU
        rgb_frame_gpu = torch.tensor(rgb_frame).to(device)

        # Detect faces using RetinaFace (RetinaFace should automatically use GPU if available)
        faces = RetinaFace.detect_faces(rgb_frame_gpu.cpu().numpy())  # Ensure input to RetinaFace is numpy array
        
        if faces:
            # Loop through each detected face and crop
            print(faces)
            for key in faces:
                face = faces[key]
                box = face['landmarks']  # Use 'facial_area' key to get the bounding box
                x1, y1, x2, y2 = [int(coord) for coord in box]
                face_image = rgb_frame[y1:y2, x1:x2]
                
                # Display the face (optional)
                plt.imshow(face_image)
                plt.axis('off')
                plt.show()
            
    # Release the video capture object
    cap.release()

print("Processing completed.")


Using device: cuda
Processing video: aagfhgtpmv.mp4


KeyboardInterrupt: 

In [16]:
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
import pywt
from scipy import stats

# Function to extract gradient features
def extract_gradient_features(gray_frame):
    gx = np.gradient(gray_frame, axis=1)
    gy = np.gradient(gray_frame, axis=0)
    gradient_magnitude = np.sqrt(gx**2 + gy**2)
    
    mean = np.mean(gradient_magnitude)
    variance = np.var(gradient_magnitude)
    skewness = stats.skew(gradient_magnitude.flatten())
    kurtosis = stats.kurtosis(gradient_magnitude.flatten())
    std_dev = np.std(gray_frame)
    
    return [mean, variance, skewness, kurtosis, std_dev]

# Function to extract GLCM features
def extract_glcm_features(gray_frame):
    # Ensure pixel values are within the range [0, 255]
    gray_frame = np.clip(gray_frame, 0, 255)
    
    # Convert to integer if necessary
    gray_frame = gray_frame.astype(np.uint8)
    
    # Number of levels for GLCM
    levels = 256  # 8-bit grayscale image has 256 levels
    
    glcm = graycomatrix(gray_frame, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=levels, symmetric=True, normed=True)
    
    contrast = graycoprops(glcm, 'contrast').mean()
    energy = graycoprops(glcm, 'energy').mean()
    homogeneity = graycoprops(glcm, 'homogeneity').mean()
    entropy = -np.sum(glcm * np.log2(glcm + 1e-10))  # Small value to avoid log(0)
    correlation = graycoprops(glcm, 'correlation').mean()
    
    return [contrast, energy, homogeneity, entropy, correlation]

# Function to extract wavelet features
def extract_wavelet_features(gray_frame):
    coeffs = pywt.wavedec2(gray_frame, 'haar', level=2)
    cH, cV, cD = coeffs[1]  # Horizontal, vertical, and diagonal coefficients
    
    features = []
    for coeff in [cH, cV, cD]:
        mean = np.mean(coeff)
        std_dev = np.std(coeff)
        energy = np.sum(coeff**2) / coeff.size
        features.extend([mean, std_dev, energy])
    
    return features

# Main function to process video and extract features
def process_video(video_path, label, output_csv):
    cap = cv2.VideoCapture(video_path)
    features_list = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Extract features
        gradient_features = extract_gradient_features(gray_frame)
        glcm_features = extract_glcm_features(gray_frame)
        wavelet_features = extract_wavelet_features(gray_frame)
        
        # Combine all features
        features = gradient_features + glcm_features + wavelet_features
        features.append(label)  # Add label (fake or real)
        
        features_list.append(features)
    
    cap.release()
    
    # Save features to CSV
    columns = ['Gradient_Mean', 'Gradient_Variance', 'Gradient_Skewness', 'Gradient_Kurtosis', 'Gray_StdDev',
               'GLCM_Contrast', 'GLCM_Energy', 'GLCM_Homogeneity', 'GLCM_Entropy', 'GLCM_Correlation',
               'Wavelet_H_Mean', 'Wavelet_H_StdDev', 'Wavelet_H_Energy',
               'Wavelet_V_Mean', 'Wavelet_V_StdDev', 'Wavelet_V_Energy',
               'Wavelet_D_Mean', 'Wavelet_D_StdDev', 'Wavelet_D_Energy', 'Label']
    
    df = pd.DataFrame(features_list, columns=columns)
    df.to_csv(output_csv, index=False)

# Example usage
process_video('F:\\deepfake-detection-challenge\\train_sample_videos\\aagfhgtpmv.mp4', 'fake', 'output_features.csv')

#F:\\deepfake-detection-challenge\\train_sample_videos\\aagfhgtpmv.mp4
